In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import gc
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()

In [2]:
%run utils.py
%run data_process.py

In [3]:
test_save=pd.read_csv('../data/test.txt',sep=" ")
test_save['model']=test_save['user_id'].apply(lambda x:x%10)
test_save['predicted_score']=0

In [381]:
i=5
data=pd.read_csv('../data/sub%d.txt'%i,sep=" ")
data=data.append(test_save[test_save.model==i]).reset_index().drop(['index'],axis=1)
data['is_trade']=data['is_trade'].fillna(0)
data = data.sort_values(by='context_timestamp').reset_index().drop(['index'],axis=1)
gc.collect()

469

In [382]:
data=data_process0(data)
data=data_process1(data)
data=data_process2(data)
data=data_process3(data)
data=data_process4(data)
data=data_process5(data)
data=data_process7(data)
data=get_peng_feature(data)

encode cate id now
fill nan now
encode now
decode time
cate_match
tongji
peng_feature
60.7159998417


In [383]:
tmp=data.groupby(['user_id','item_id'],as_index=False)['user_item_cumcount'].max().rename(columns={'user_item_cumcount':'user_item_maxrank'})
data=data.merge(tmp,on=['user_id','item_id'],how='left')
data['user_item_last']=(data['user_item_cumcount']==data['user_item_maxrank'])

In [384]:
day={7:data[data.day==7].copy()}

In [385]:
count_list=[]
sum_list=[]
'''
统计7天之内的点击数,购买数，反映热度
'''
for cols in [['shop_id'],['item_id'],['user_id'],['item_brand_id'],['item_city_id'],
             ['user_id','item_id'],['user_id','shop_id'],['user_id','item_brand_id'],['user_id','item_city_id']]:
    if len(cols)==1:
        name1='cnt_'+cols[0]
        name2='sum_'+cols[0]
    else:
        name1='cnt_'+cols[0]+'_'+cols[1]
        name2='sum_'+cols[0]+'_'+cols[1]
    if name1 in day[7].columns:
        del day[7][name1]
    if name2 in day[7].columns:
        del day[7][name2]
        
    day[7]=day[7].merge(get_silde_cnt2(data,7,cols,name1,7),on=cols,how='left')
    day[7][name1]=day[7][name1].fillna(0)
    day[7]=day[7].merge(get_silde_sum2(data,7,cols,name2,7),on=cols,how='left')
    day[7][name2]=day[7][name2].fillna(0)
    gc.collect()
    count_list+=[name1]
    sum_list+=[name2]
print count_list,sum_list

['cnt_shop_id', 'cnt_item_id', 'cnt_user_id', 'cnt_item_brand_id', 'cnt_item_city_id', 'cnt_user_id_item_id', 'cnt_user_id_shop_id', 'cnt_user_id_item_brand_id', 'cnt_user_id_item_city_id'] ['sum_shop_id', 'sum_item_id', 'sum_user_id', 'sum_item_brand_id', 'sum_item_city_id', 'sum_user_id_item_id', 'sum_user_id_shop_id', 'sum_user_id_item_brand_id', 'sum_user_id_item_city_id']


In [389]:
day[7]['item_user_ratio']=day[7]['cnt_user_id_item_id']/day[7]['cnt_user_id']
day[7]['shop_user_ratio']=day[7]['cnt_user_id_shop_id']/day[7]['cnt_user_id']
day[7]['brand_user_ratio']=day[7]['cnt_user_id_item_brand_id']/day[7]['cnt_user_id']
day[7]['user_item_cumcount2']=day[7]['user_item_cumcount']/day[7]['user_item_maxrank']
day[7]=get_count(day[7],['user_id'],'day7_user_cnt','instance_id')

In [397]:
if 'cnt_user_id' in count_list:
    count_list.remove('cnt_user_id')
    sum_list.remove('sum_user_id')
ratio_list=['item_user_ratio','shop_user_ratio','brand_user_ratio']
peng_feature=['user_count_10_bf','user_count_10_af','user_shop_count_10_bf','user_shop_count_10_af','user_item_count_10_bf','user_item_count_10_af']
cate_feature=['user_gender_id','user_occupation_id','cate_id']
features=[ 'item_price_level', 'item_sales_level','item_collected_level','item_pv_level','user_age_level', 'user_star_level',
         'shop_review_num_level', 'shop_star_level','shop_review_positive_rate', 'shop_score_service','shop_score_delivery', 
        'shop_score_description','cate_match','context_page_id']+\
        [ 'shop_item_nunique','user_price_rank','user_sale_rank','user_collect_rank','user_shop_cumcount','user_item_cumcount',
        'price_rank','sale_rank','collect_rank']+\
        cate_feature+count_list+sum_list+ratio_list+peng_feature+['user_item_last','user_item_cumcount2','day7_user_cnt']
target='is_trade'
print len(features)

72


In [392]:
filter_day7=(day[7].hour>9)
train=day[7][(day[7].model.isnull())&(~filter_day7)].copy().reset_index().drop('index',axis=1)
val=day[7][(day[7].model.isnull())&(filter_day7)].copy().reset_index().drop('index',axis=1)
test=day[7][day[7].model.notnull()].copy().reset_index().drop('index',axis=1)
print train.shape,val.shape,test.shape
gc.collect()

(80685, 107) (26897, 107) (51766, 107)


157

In [398]:
gbm = lgb.LGBMClassifier(objective='binary',
                                max_depth=5,
                                num_leaves=16,
                                learning_rate=0.03,
                                n_estimators=2000,
                                colsample_bytree=0.3,#0.3
                                subsample = 0.75,#0.75
                                n_jobs=4,
                                lambda_l2=10,
                                seed=2018
                                )
gbm.fit(train[features],train[target],categorical_feature=cate_feature,
                    eval_set=[(val[features],val[target])],
                    eval_metric='binary_logloss',
                    early_stopping_rounds=40,verbose=40)

val_error=gbm.best_score_['valid_0']['binary_logloss']
best_model=gbm.booster_
train_pred=best_model.predict(train[features], num_iteration=best_model.best_iteration)
print val_error,log_loss(train[target],train_pred)

Training until validation scores don't improve for 40 rounds.
[40]	valid_0's binary_logloss: 0.280931
[80]	valid_0's binary_logloss: 0.190626
[120]	valid_0's binary_logloss: 0.172231
[160]	valid_0's binary_logloss: 0.167255
[200]	valid_0's binary_logloss: 0.165326
[240]	valid_0's binary_logloss: 0.164458
[280]	valid_0's binary_logloss: 0.164039
[320]	valid_0's binary_logloss: 0.163761
[360]	valid_0's binary_logloss: 0.163578
[400]	valid_0's binary_logloss: 0.163523
[440]	valid_0's binary_logloss: 0.163482
[480]	valid_0's binary_logloss: 0.163469
Early stopping, best iteration is:
[471]	valid_0's binary_logloss: 0.163451
0.163450803242 0.165589676499
